# 抽出型文書要約

## 手続き

1. 前準備
1. クリーニング(文区切り，絵文字/記号などの削除)
1. 文章の正規化(全角半角変換，数字の削除)
1. 文章の単語分割(形態素解析)
1. 単語の正規化(同じ意味を持つ単語の統一)
1. ストップワードの除去(名詞/形容詞/副詞/動詞のみの利用，辞書による除去)
1. 文書のベクトル表現(TF-IDF)
1. 要約モデルの適用(LexRank)
1. FlaskでAPIに

## 実装

### 1. 前準備

In [41]:
import numpy as np
import pandas as pd

import neologdn

# 文区切り
import functools
from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

# クリーニング
import re
from modules import data_cleaning as dc

# MeCab(形態素解析)
import MeCab as mecab
mecab = MeCab.Tagger('-r /etc/mecabrc -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

# 単語の正規化
import emoji
import mojimoji

# ストップワード除去
# 名詞/形容詞/副詞/動詞のみを抽出できるライブラリのインポート
import mecabpr
mpr = mecabpr.MeCabPosRegex('-r /etc/mecabrc -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

# 抽出型要約モデル(LexRank)
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

### 2. クリーニング

In [58]:
split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_te = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)([\r\n]+)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_te, split_punc2)

text_org = """
項目gp2gp3耐久性99.9%99.9%IOPS100 IOPS ~ 16,000 IOPS 3,000 IOPS ~ 16,000 IOPSスループット〜 250 MiB/秒 250 MiB/秒 〜 1,000 MiB/秒 ストレージ料金 （月あたり、1GBあたり、東京リージョン）$0.12 $0.096 IOPS料 金（月あたり、1GBあたり、東京リージョン）- 3,000 IOPSまで無料 、 $0.006/IOPS （3,000IOPS超える場合）スループット料金（月あたり、1GBあたり、東京リージョン）- 125 MB/秒まで無料 、 $0.048 MB/s （125 MB/sを超える場合） gp2はボリュームサイズによってIOPSが割り当てられ、ベースラインパフォーマンスで100IOPSが保証されておりました。
gp3は 3,000IOPS がベースラインとして保証され、さらにそれ以上のIOPSが求められる場合は追加料金を払うことで最大16,000IOPSまで拡張できます。
また、追加料金を支払うことで個別にスループットとIOPSを拡張できますので、アプリケーション要件に合わせて使用することができます。
ベースラインパフォーマンスは大幅に上がっているので、これまでのように不要なボリューム追加やプロビジョンドIOPSを使わなくて済むケースが増えると思います。
"""

In [59]:
# 『楽しみ〜〜〜』の『〜〜〜』などを削除
text = neologdn.normalize(text_org)

In [60]:
sentences = list(segmenter(text))
pd.DataFrame(sentences)

,0
0,"項目gp2gp3耐久性99.9%99.9%IOPS100 IOPS 16,000 IOPS ..."
1,"gp3は3,000IOPSがベースラインとして保証され、さらにそれ以上のIOPSが求められる..."
2,また、追加料金を支払うことで個別にスループットとIOPSを拡張できますので、アプリケーション...
3,ベースラインパフォーマンスは大幅に上がっているので、これまでのように不要なボリューム追加やプ...


### 3. 文章の正規化

In [61]:
sentences_dc = dc.data_cleaning(sentences)
pd.DataFrame(sentences_dc)

,0
0,項目gp0gp0耐久性0iops0iops0iops0iops0iopsスループット0mib...
1,gp0は0iopsがベースラインとして保証され、さらにそれ以上のiopsが求められる場合は追...
2,また、追加料金を支払うことで個別にスループットとiopsを拡張できますので、アプリケーション...
3,ベースラインパフォーマンスは大幅に上がっているので、これまでのように不要なボリューム追加やプ...


### 4.5.6. 文章の単語分割，単語の正規化，ストップワード除去

In [63]:
sentence_words = []
result = []
for i, sentence in enumerate(sentences_dc):
    # 名詞|形容詞|副詞|動詞のみの抽出
    ma = sum(mpr.findall(sentence, "(名詞|形容詞|副詞|動詞)", raw=True), [])
    # 活用形の統一(基本形へ)
    sentence_ma = []
    for word in ma:
        if word.split(',')[6] == "*":
            if not bool(re.search(r'[a-zA-Z]',word.split('\t')[0])):
                sentence_ma.append(word.split('\t')[0])
            else :
                sentence_ma.append(re.sub(r"[a-zA-Z]", "", word.split('\t')[0]))
                    
        elif bool(re.search(r'[a-zA-Z]',word.split(',')[6])):
            if not bool(re.search(r'[a-zA-Z]',word.split(',')[7])):
                sentence_ma.append(word.split(',')[7])
            else :
                sentence_ma.append(re.sub(r"[a-zA-Z]", "", word.split(',')[7]))
        else:
            sentence_ma.append(word.split(',')[6])
    result.append(sentence_ma)

In [64]:
result

[['項目',
  'ゲーペー',
  'ゼログラム',
  '',
  '0',
  '耐久性',
  '0',
  'アイオーピーエス',
  '0',
  'アイオーピーエス',
  '0',
  'アイオーピーエス',
  '0',
  'アイオーピーエス',
  '0',
  'アイオーピーエス',
  'スループット',
  'ゼロメートル',
  'イヴ',
  '秒',
  'ゼロメートル',
  'イヴ',
  '秒',
  'ゼロメートル',
  'イヴ',
  '秒',
  'ストレージ',
  '料金',
  '月',
  'あたり',
  'ゼログラム',
  '',
  'あたり',
  '東京',
  'リージョン',
  '0',
  'アイオーピーエス',
  '料金',
  '月',
  'あたり',
  'ゼログラム',
  '',
  'あたり',
  '東京',
  'リージョン',
  '0',
  'アイオーピーエス',
  '無料',
  '0',
  'アイオーピーエス',
  '0',
  'アイオーピーエス',
  '超える',
  '場合',
  'スループット',
  '料金',
  '月',
  'あたり',
  'ゼログラム',
  '',
  'あたり',
  '東京',
  'リージョン',
  'ゼロメートル',
  '',
  '秒',
  '無料',
  'ゼロメートル',
  'バスストップ',
  'ゼロメートル',
  'バスストップ',
  '超える',
  '場合',
  'ゲーペー',
  '0',
  'ボリューム',
  'サイズ',
  'アイオーピーエス',
  '割り当てる',
  'られる',
  'ベースラインパフォーマンス',
  '0',
  'アイオーピーエス',
  '保証',
  'する',
  'れる',
  'おる'],
 ['ゲーペー',
  '0',
  '0',
  'アイオーピーエス',
  'ベース',
  'ライン',
  '保証',
  'する',
  'れる',
  'さらに',
  'それ',
  '以上',
  'アイオーピーエス',
  '求める',
  'られる',
  '場合',
  '追加',
  '料金',
  '払う',
 

In [65]:
corpus = [' '.join(sentence) + u'。' for sentence in result]
"".join(corpus)

'項目 ゲーペー ゼログラム  0 耐久性 0 アイオーピーエス 0 アイオーピーエス 0 アイオーピーエス 0 アイオーピーエス 0 アイオーピーエス スループット ゼロメートル イヴ 秒 ゼロメートル イヴ 秒 ゼロメートル イヴ 秒 ストレージ 料金 月 あたり ゼログラム  あたり 東京 リージョン 0 アイオーピーエス 料金 月 あたり ゼログラム  あたり 東京 リージョン 0 アイオーピーエス 無料 0 アイオーピーエス 0 アイオーピーエス 超える 場合 スループット 料金 月 あたり ゼログラム  あたり 東京 リージョン ゼロメートル  秒 無料 ゼロメートル バスストップ ゼロメートル バスストップ 超える 場合 ゲーペー 0 ボリューム サイズ アイオーピーエス 割り当てる られる ベースラインパフォーマンス 0 アイオーピーエス 保証 する れる おる。ゲーペー 0 0 アイオーピーエス ベース ライン 保証 する れる さらに それ 以上 アイオーピーエス 求める られる 場合 追加 料金 払う こと 最大 0 アイオーピーエス 拡張 できる。追加 料金 支払う こと 個別 スループット アイオーピーエス 拡張 できる アプリケーション 要件 合わせる 使用 する こと できる。ベースラインパフォーマンス 大幅 上がる いる これ よう 不要 ボリューム 追加 プロビジョンド アイオーピーエス 使う 済む ケース 増える 思う。'

In [66]:
corpus

['項目 ゲーペー ゼログラム  0 耐久性 0 アイオーピーエス 0 アイオーピーエス 0 アイオーピーエス 0 アイオーピーエス 0 アイオーピーエス スループット ゼロメートル イヴ 秒 ゼロメートル イヴ 秒 ゼロメートル イヴ 秒 ストレージ 料金 月 あたり ゼログラム  あたり 東京 リージョン 0 アイオーピーエス 料金 月 あたり ゼログラム  あたり 東京 リージョン 0 アイオーピーエス 無料 0 アイオーピーエス 0 アイオーピーエス 超える 場合 スループット 料金 月 あたり ゼログラム  あたり 東京 リージョン ゼロメートル  秒 無料 ゼロメートル バスストップ ゼロメートル バスストップ 超える 場合 ゲーペー 0 ボリューム サイズ アイオーピーエス 割り当てる られる ベースラインパフォーマンス 0 アイオーピーエス 保証 する れる おる。',
 'ゲーペー 0 0 アイオーピーエス ベース ライン 保証 する れる さらに それ 以上 アイオーピーエス 求める られる 場合 追加 料金 払う こと 最大 0 アイオーピーエス 拡張 できる。',
 '追加 料金 支払う こと 個別 スループット アイオーピーエス 拡張 できる アプリケーション 要件 合わせる 使用 する こと できる。',
 'ベースラインパフォーマンス 大幅 上がる いる これ よう 不要 ボリューム 追加 プロビジョンド アイオーピーエス 使う 済む ケース 増える 思う。']

### 7. 8.文書のベクトル表現，要約モデル(Lexrank)の適用

In [70]:
# 文章要約メソッド
def summy_test(sentences_org, corpus, sum_count):
    sentences = [' '.join(sentence) + u'。' for sentence in corpus]
    text_prep = "".join(sentences)
    parser = PlaintextParser.from_string(text_prep, Tokenizer('japanese'))

    summarizer = LexRankSummarizer()
    summarizer.stop_words = ['']
    # sentences_countにて、sentence（文章の数）を選びます。
    # 例えばsentences_countを３にすると文章を3文に要約してくれます。
    summary = summarizer(document=parser.document, sentences_count=sum_count)
    
    print(u'文書要約結果')    
    b = []
    for sentence in summary:
        b.append(sentences_org[sentences.index("{}".format(sentence.__str__()))])
        b.append("\n")
    return "".join(b)

In [71]:
print(summy_test(sentences, result, sum_count=2))

(<Sentence: ゲーペー 0 0 アイオーピーエス ベース ライン 保証 する れる さらに それ 以上 アイオーピーエス 求める られる 場合 追加 料金 払う こと 最大 0 アイオーピーエス 拡張 できる。>, <Sentence: ベースラインパフォーマンス 大幅 上がる いる これ よう 不要 ボリューム 追加 プロビジョンド アイオーピーエス 使う 済む ケース 増える 思う。>)
文書要約結果
gp3は3,000IOPSがベースラインとして保証され、さらにそれ以上のIOPSが求められる場合は追加料金を払うことで最大16,000IOPSまで拡張できます。
ベースラインパフォーマンスは大幅に上がっているので、これまでのように不要なボリューム追加やプロビジョンドIOPSを使わなくて済むケースが増えると思います。



### 9. モジュールの動作確認

In [69]:
%load_ext autoreload
%autoreload 2
from modules import extractive_summarization as es

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
text = """
ジョン・ロバーツ連邦最高裁判所長官、カーター大統領、クリントン大統領、ブッシュ大統領、オバマ大統領、アメリカ国民の皆さん、そして、世界中の皆さん、ありがとうございます。

私たちアメリカ国民は今、素晴らしい国家的な努力に参加し、国を再建して、すべての人のために約束を果たします。私たちは共に、アメリカの、そして、世界の歩む道を決めるのです。これから歩む長い道です。私たちは課題に直面するでしょう。さまざなま困難にも直面するでしょう。しかし、その仕事をやり遂げます。

4年ごとに、私たちはこうした道のりのために集まり、秩序だって速やかに政権を移行します。この政権移行を快く支えてくれたオバマ大統領とミシェル・オバマ大統領夫人に感謝します。素晴らしい人たちです。ありがとうございます。

本日の式典には、とても特別な意味があります。なぜなら、ひとつの政権から別の政権へ、または、ひとつの政党から別の政党へ、単なる政権交代をしているわけではなく、ワシントンD.C.から国民である皆さんへ、政権を取り戻しているからです。

あまりにも長い間、ワシントンにいる一部の人たちだけが、政府から利益や恩恵を受けてきました。その代償を払ったのは国民です。ワシントンは繁栄しましたが、国民はその富を共有できませんでした。政治家は潤いましたが、職は失われ、工場は閉鎖されました。権力層は自分たちを守りましたが、アメリカ市民を守りませんでした。彼らの勝利は、皆さんの勝利ではありませんでした。彼らは首都ワシントンで祝福しましたが、アメリカ全土で苦しんでいる家族への祝福は、ほとんどありませんでした。

すべての変革は、この場所から始まります。今、ここで始まっているのです。なぜなら、この瞬間は皆さんの瞬間だからです。皆さんのものです。今日、ここに集まっている皆さん、アメリカ中でこれを見ている皆さんのものです。今日という日は、皆さんの1日なのです。これは皆さんの式典です。そして、このアメリカ合衆国は、皆さんの国なのです。

本当に大切なことは、どの党が政権を握るかということではなく、政府が国民により統治されることです。2017年1月20日は、国民がこの国の治める日として、これからずっと記憶に刻まれるでしょう。この国の忘れ去られた人々は、もう忘れ去られることはありません。誰もが皆さんに耳を傾けています。何千万の人々が、歴史的な運動に参加しています。

今まで世界が見たことのない動きが起きています。この動向の中心にあるのは、とても強い信念です。それは、国は国民に奉仕するために存在しているということです。アメリカ国民は、子供たちのために素晴らしい学校を望んでいます。また、家族のために安全を、自分自身のために良い仕事を望んでいます。正しい人々、そして、正しい国民がそう望むのは正当で、当然のことです。

しかし、多くの市民には、異なる現実が存在しています。母親と子供は都市部で貧困に苦しみ、工場は錆びき、アメリカ中に墓石のごとく散らばっています。教育は高額で、若く輝かしい生徒たちは、知識を習得できていません。犯罪、ギャング、麻薬があまりにも多くの命を奪い、花開くことのない可能性をこの国から奪っています。

こうしたアメリカの殺戮は、今ここで終わります。今、ここでです。

私たちはひとつの国家であり、彼らの痛みは、私たちの痛みです。彼らの夢は、私たちの夢です。そして、彼らの成功は、私たちの成功です。私たちは、ひとつの心、ひとつの故郷、ひとつの輝きに満ちた運命を共有しています。今日、私がした就任の誓いは、すべてのアメリカ国民の忠誠の誓いです。

何十年もの間、私たちはアメリカの産業を犠牲にし、外国の産業を豊かにしてきました。他の国々の軍隊を援助してきました。一方で、アメリカの軍隊は、悲しくも枯渇しています。私たちは他の国の国境を守っていますが、自分たちの国境を守るのを拒んでいます。海外に数兆ドルを投資しましたが、アメリカのインフラは絶望に陥り、腐っています。他の国々を豊かにしましたが、自国の富、力、自信は、地平線のかなたへ消えて行きました。ひとつずつ、工場が閉鎖され、この国を去りました。数百万人のアメリカ人労働者が置き去りになることなど考えもしないで、そうしたのです。中間層の富が、その家庭から奪われ、世界中に再分配されました。

しかし、それは過去です。今、私たちは未来だけを見据えています。私たちは今日、ここに集まり、新しい決意を発し、すべての街、すべての外国の首都、すべての政権にそれを響かせます。今日、この日から始まります。新しいビジョンがアメリカを治めるでしょう。今日、この日から、アメリカ第一のみになります。アメリカ第一です。

貿易、税金、移民、外交についてのすべての決定は、アメリカの労働者と家族の利益のために下されます。他国の暴挙から国境を守らなければなりません。彼らは私たちの商品を生産し、私たちの会社を盗み、私たちの仕事を破壊しています。保護こそが偉大な繁栄と力に繋がるのです。

私は全力で皆さんのために戦います。決して失望させません。アメリカは再び勝利します。これまでにない勝利です。雇用を取り戻し、国境を回復し、富を取り戻し、そして、夢を取り戻します。このすばらしい国の隅々に新しい道路、橋、空港、トンネル、鉄道を建設します。生活保護を受けている人たちに仕事を与え、アメリカの労働者の手と力で国を再建します。

私たちは2つの単純なルールに従います。アメリカ製の商品を買い、アメリカ人を雇うことです。世界の国々と友好的な善意の関係を築きますが、すべての国には自国の利益を優先させる権利があることを理解した上で、そうします。私たちは自分たちの生き方をすべての人に押し付けることはしませんが、模範として輝やかせたいと思っています。私たちはすべての人が追随するような輝きを放つでしょう。私たちは古い同盟関係を強化し、新たなものを形づくります。イスラム過激派のテロに対し世界を結束させ、地球上から完全に根絶させます。

私たちの政治の基盤は、アメリカ合衆国への完全な忠誠心です。国への忠誠を通し、私たちはお互いへの忠誠を再発見するでしょう。愛国心に心を開けば、偏見など持たないはずです。聖書はこう教えています。神の民が一体となって暮らすのは、何と素晴らしく喜ばしいことでしょう、と。私たちは隠さずに思っていることを語り、相違について討論しますが、いつも団結を求めなければなりません。アメリカが団結すれば、誰もアメリカを止めることはできません。

恐れることはありません。私たちは守られています。そして、私たちはこの先も守られるでしょう。私たちは軍や法執行機関の素晴らしい人たちに守られるています。そして、最も大切なのは、神により守られていることです。

最後に、私たちは大きく考え、さらに大きな夢をみなければなりません。アメリカで、私たちは分かっていると思うのですが、国家は、努力してこそ存続するのです。口ばかりで行動が伴わない政治家をこれ以上受け入れることはできません。彼らは文句ばかり言って、何もしていません。意味のないお喋りは終わりを迎える時です。今、行動の時が来ています。それはできない、と言うのはやめましょう。どんな課題も、心を開き、戦い、アメリカの精神を持てば、乗り越えられます。失敗することはありません。私たちの国は再び繁栄し、栄えるでしょう。

私たちは、新しい時代の誕生に立ち会っています。宇宙の神秘を解き明かし、地球上から病気の苦しみを失くし、未来の産業とテクノロジーを利用する準備をしています。新しいアメリカの誇りは、私たちの魂を揺さぶり、視野を高め、分断を埋めるでしょう。今こそ、思い出す時です。兵士が永遠に心に刻む知恵です。黒い肌、褐色の肌、白い肌、誰であろうと、同じ愛国心の赤い血が流れています。私たちは同じ輝かしい自由を享受しています。みんな同じ偉大な星条旗に忠誠を誓っているのです。子供がデトロイトの都市部で生まれようと、ネブラスカの風の吹く平原で生まれようと、同じ夜空を見上げ、同じ夢を心に抱き、同じ全知全能の創造主によって生命の息吹が吹き込まれます。

ですから、アメリカ国民の皆さん、すべての街に住んでいる市民の皆さん、それが近くても、遠くても、小さくても、大きくても、山から山まで、海から海まで、この言葉を聞いてください。皆さんは再び無視されることは決してありません。皆さんの声、希望、夢が、アメリカの歩む道を決めるのです。そして、皆さんの勇気、善意、愛が、その道を永遠に照らすのです。

一致団結して、私たちはアメリカを再び強い国にします。アメリカを再び富める国にします。アメリカを再び誇り高い国にします。アメリカを再び安全な国にします。そうです。ともに力を合わせ、アメリカを再び偉大な国にします。ありがとうございます。皆さんに神の祝福がありますように。そして、アメリカに神の祝福がありますように。ありがとうございます。アメリカに神の祝福あれ。
"""

In [56]:
summary = es.preprocessed_lexrank(text, sum_count=10)

In [57]:
print(summary)

私たちアメリカ国民は今、素晴らしい国家的な努力に参加し、国を再建して、すべての人のために約束を果たします。
ワシントンは繁栄しましたが、国民はその富を共有できませんでした。
皆さんのものです。
今日、ここに集まっている皆さん、アメリカ中でこれを見ている皆さんのものです。
そして、このアメリカ合衆国は、皆さんの国なのです。
それは、国は国民に奉仕するために存在しているということです。
私たちは今日、ここに集まり、新しい決意を発し、すべての街、すべての外国の首都、すべての政権にそれを響かせます。
彼らは私たちの商品を生産し、私たちの会社を盗み、私たちの仕事を破壊しています。
私たちは守られています。
皆さんは再び無視されることは決してありません。

